In [1]:
import os
import time
import numpy as np
import tensorflow as tf

In [2]:
path_to_file=tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)

1115394/1115394 [==============================] - 0s 0us/step


In [4]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


In [7]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [8]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

65 unique characters


In [9]:
example_text=["abcdefg", "xyz"]
chars=tf.strings.unicode_split(example_text, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [10]:
ids_from_chars=tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

In [11]:
ids=ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [12]:
chars_from_ids=tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [13]:
chars=chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [14]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [15]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [16]:
all_ids=ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1], dtype=int64)>

In [17]:
ids_Dataset=tf.data.Dataset.from_tensor_slices(all_ids)

In [18]:
for ids in ids_Dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i
t
i


In [19]:
seq_length=100
examples_per_epoch=len(text)//(seq_length+1)

In [20]:
sequences=ids_Dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [21]:
for seq in sequences.take(1):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [24]:
def split_input_target(sequence):
    input_text=sequence[:-1]
    target_text=sequence[1:]
    return input_text, target_text

In [25]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [26]:
dataset=sequences.map(split_input_target)

In [28]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [29]:
BATCH_SIZE=64
BUFFER_SIZE=10000

dataset=(
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [30]:
vocab_size=len(vocab)

embedding_dim=256

rnn_units=1024

In [31]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding=tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru=tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        self.dense=tf.keras.layers.Dense(vocab_size)
        
    def call(self, inputs, states=None, return_state=False, training=False):
        x=self.embedding(inputs, training=training)
        if states is None:
            states=self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x=self.dense(x, training=training)
        if return_state:
            return x, states
        else:
            return x

In [32]:
model=MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [33]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions=model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [34]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4,022,850
Trainable params: 4,022,850
Non-trainable params: 0
_________________________________________________________________


In [35]:
sampled_indices=tf.random.categorical(
    example_batch_predictions[0], num_samples=1
)
sampled_indices=tf.squeeze(sampled_indices, axis=-1).numpy()

In [36]:
sampled_indices

array([30, 23, 27, 56, 36, 16, 19,  7,  2, 10, 26, 23, 17, 30, 29, 62, 42,
       20, 44, 54,  9, 10, 51, 36, 20, 28, 19,  6, 36, 22,  4, 61, 20, 64,
       45, 64, 47, 54, 26, 60, 62, 39, 47,  4, 41,  3, 53, 30, 18, 27, 25,
       39,  5,  5, 45, 20, 61, 32, 33, 33, 33, 57, 43, 32, 17, 25, 22, 16,
        7, 35, 32, 36, 35, 58, 13,  2, 49, 40,  6, 18, 22, 15, 39, 36, 47,
       10, 26, 56, 28, 26, 44, 32, 12, 65, 43, 15,  0, 13, 16, 39],
      dtype=int64)

In [37]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b', resides this dejected Mariana. At that\nplace call upon me; and dispatch with Angelo, that\nit may b'

Next Char Predictions:
 b"QJNqWCF, 3MJDQPwcGeo.3lWGOF'WI$vGyfyhoMuwZh$b!nQENLZ&&fGvSTTTrdSDLIC,VSWVs? ja'EIBZWh3MqOMeS;zdB[UNK]?CZ"


In [38]:
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [39]:
example_batch_mean_loss=loss(target_example_batch, example_batch_predictions)
print(
    "Prediction Shape: ",
    example_batch_predictions.shape,
    "# (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:     ",example_batch_mean_loss)

Prediction Shape:  (64, 100, 66) # (batch_size, sequence_length, vocab_size)
Mean loss:      tf.Tensor(4.1891365, shape=(), dtype=float32)


In [40]:
tf.exp(example_batch_mean_loss).numpy()

65.965805

In [41]:
model.compile(optimizer='adam', loss=loss)

In [42]:
checkpoint_dir="./training_checkpoints"
checkpoint_prefix=os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

In [50]:
EPOCHS=30
history=model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30


172/172 [==============================] - 20s 109ms/step - loss: 1.1554
Epoch 2/30
172/172 [==============================] - 20s 108ms/step - loss: 1.1137
Epoch 3/30
172/172 [==============================] - 20s 110ms/step - loss: 1.0687
Epoch 4/30
172/172 [==============================] - 20s 110ms/step - loss: 1.0218
Epoch 5/30
172/172 [==============================] - 20s 109ms/step - loss: 0.9723
Epoch 6/30
172/172 [==============================] - 20s 110ms/step - loss: 0.9194
Epoch 7/30
172/172 [==============================] - 20s 109ms/step - loss: 0.8663
Epoch 8/30
172/172 [==============================] - 20s 110ms/step - loss: 0.8135
Epoch 9/30
172/172 [==============================] - 21s 112ms/step - loss: 0.7620
Epoch 10/30
172/172 [==============================] - 20s 111ms/step - loss: 0.7141
Epoch 11/30
172/172 [==============================] - 20s 111ms/step - loss: 0.6712
Epoch 12/30
172/172 [==============================] - 21s 112ms/step - loss: 0.6318


In [51]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature=temperature
        self.model=model
        self.chars_from_ids=chars_from_ids
        self.ids_from_chars=ids_from_chars
        
        skip_ids=self.ids_from_chars(['[UNK]'])[:,None]
        sparse_mask=tf.SparseTensor(
            values=[-float("inf")]*len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        
        self.prediction_mask=tf.sparse.to_dense(sparse_mask)
        
    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars=tf.strings.unicode_split(inputs, "UTF-8")
        input_ids=self.ids_from_chars(input_chars).to_tensor()
        
        predicted_logits, states=self.model(
            inputs=input_ids, states=states, return_state=True
        )
        
        predicted_logits=predicted_logits[:, -1, :]
        predicted_logits=predicted_logits/self.temperature
        predicted_logits=predicted_logits+self.prediction_mask
        
        predicted_ids=tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids=tf.squeeze(predicted_ids, axis=-1)
        
        predicted_chars=self.chars_from_ids(predicted_ids)
        return predicted_chars, states

In [52]:
one_step_model=OneStep(model, chars_from_ids, ids_from_chars)

In [53]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
The excepprog's man is a pitician of great
But doth become my wife in his grace.

First Gentleman:
Well, my lord: welcome, gladuous do answer there.

LUCIO:
Good Bolingbrok,
So long a-monity will be bruitful victory.
Is not the queen who stand now with rain's ta'en?

First Servant:
What's honest tuated shall be that of Christen of the earth,
The generous and unperitice.
What, ho, how but we omit,
When thou hast have tress'd me of this defend.
I will, my noble cousin; from him proud, and
would pluck him conceallied accusans: God, which I please,
Unworthy thee, fond worthy widows,
And make me in his master, but through the world so side;
For their advices, tirrahts with wolves are of such
deeps, from out a world as much belosed:
More than our faults, I hold thee better:
By heaven, I fear, not for my company.

LADY GREY:
No, man; son, and my love.
When this is lost a body for battle;
And here I take them such a sin.

KING HENRY VI:
I swear I'll crave the brother of your lasting whe

In [54]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
Ay, as your reigning men have done unto his
prayers. O, the king's son,
Who, with a grandar kingly charities that I was not queen.

PETRUCHIO:
I say it is remembar.

GLOUCESTER:
Come now, come by: almost all these powers
Apoperity on our heart: these are the spoilest
so vouchish part off together to the day,
So would have setted: hanging in the trests ones, you will te
dagger-in-lew-ply of thy scaleful wedding-day.

HENRY BOLINGBROKE:
Lord marshal, let thee jot! Come you to some hand:
What if become my life upon this hother,
Not your consent, if you should not be avoided,
Stand by unlikely turn to Henry.
Please you to do't at place?

JULIET:
Thy body shall be placed in their death at gate
And use your manners disperse that bawdly change,
so.' farewell, and still too good father, bring it there.

ROMEO:
Gream of Naples, pity'd him, the sons of oathful evils,
That is not made of Angelo. if my resolve
I never yet make men earth.
High-selves---beat not at all as yonder already:
My u